# Converting to number column types with to_numeric

## Example #1

In [17]:
import dask
import dask.dataframe as dd
import pandas as pd

In [18]:
df = pd.DataFrame(
    {"nums": [1, 2.8, 3, 4, "hi", 6], "letters": ["a", "b", "c", "d", "e", "f"]}
)
ddf = dd.from_pandas(df, npartitions=2)

In [20]:
print(ddf.compute())

  nums letters
0    1       a
1  2.8       b
2    3       c
3    4       d
4   hi       e
5    6       f


In [21]:
ddf.dtypes

nums       object
letters    object
dtype: object

## Option 1

In [47]:
ddf["nums"] = ddf["nums"].astype("float64")

In [48]:
ddf.dtypes

nums       float64
letters     object
dtype: object

In [49]:
print(ddf.compute())

ValueError: could not convert string to float: 'hi'

## Option 2

In [52]:
print(ddf.compute())

  nums letters
0    1       a
1  2.8       b
2    3       c
3    4       d
4   hi       e
5    6       f


In [53]:
ddf.dtypes

nums       object
letters    object
dtype: object

In [54]:
def try_to_float(something):
    try:
        return float(something)
    except (ValueError, TypeError):
        return None

In [55]:
ddf1 = ddf.assign(nums = ddf.nums.apply(try_to_float, meta=('object', 'float64')))

In [56]:
ddf1.dtypes

nums       float64
letters     object
dtype: object

In [57]:
print(ddf1.compute())

   nums letters
0   1.0       a
1   2.8       b
2   3.0       c
3   4.0       d
4   NaN       e
5   6.0       f


## Option 3

In [22]:
print(ddf.compute())

  nums letters
0    1       a
1  2.8       b
2    3       c
3    4       d
4   hi       e
5    6       f


In [23]:
ddf["nums"] = dd.to_numeric(ddf["nums"], errors="coerce")

In [24]:
ddf.dtypes

nums        int64
letters    object
dtype: object

In [26]:
print(ddf.compute())

   nums letters
0   1.0       a
1   2.8       b
2   3.0       c
3   4.0       d
4   NaN       e
5   6.0       f


## Example #2

In [51]:
df2 = pd.DataFrame(
    {"n1": ["bye", 2.8, 3], "n2": ["7.7", "8", 9.2]}
)
ddf2 = dd.from_pandas(df, npartitions=2)

In [52]:
print(ddf2.compute())

    n1   n2
0  bye  7.7
1  2.8    8
2    3  9.2


In [53]:
ddf2.dtypes

n1    object
n2    object
dtype: object

In [54]:
ddf2["n2"] = ddf2["n2"].astype("float64")

In [55]:
ddf2.dtypes

n1     object
n2    float64
dtype: object

In [35]:
print(ddf2.compute())

    n1   n2
0  bye  7.7
1  2.8  8.0
2    3  9.2


In [36]:
ddf2["n1"] = ddf2["n1"].astype("float64")

In [37]:
print(ddf2.compute())

ValueError: could not convert string to float: 'bye'

In [47]:
ddf2["n1"] = dd.to_numeric(ddf["n1"])

In [48]:
ddf2.compute()

ValueError: Unable to parse string "bye" at position 0

In [56]:
ddf2["n1"] = dd.to_numeric(ddf["n1"], errors="coerce")

In [57]:
print(ddf2.compute())

    n1   n2
0  NaN  7.7
1  2.8  8.0
2  3.0  9.2
